# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
# !pip install XlsxWriter

In [41]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
pd.set_option('display.max_columns', None)

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [6]:
IEX_CLOUD_API_TOKEN = 'Tpk_a72dc9cdfcb047559677a5a1777139d7'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [20]:
symbol ='appl'
api_url = f'https://sandbox.iexapis.com/stable/stock/aapl/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [29]:
%pprint
data

Pretty printing has been turned OFF


{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'K/L(AQGA NEAAT SE)DNCT MOLLSGEBSR', 'calculationPrice': 'close', 'open': 137.41, 'openTime': 1651749304764, 'openSource': 'ailficof', 'close': 138.06, 'closeTime': 1648882316378, 'closeSource': 'iflaocfi', 'high': 140.2, 'highTime': 1638972723839, 'highSource': 'taIde EXLas Tr', 'low': 135.578, 'lowTime': 1617036403679, 'lowSource': 'audep5emryleed1t  ci in', 'latestPrice': 135.89, 'latestSource': 'Close', 'latestTime': 'February 12, 2021', 'latestUpdate': 1615378590936, 'latestVolume': 62262910, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 140.06, 'delayedPriceTime': 1678872331375, 'oddLotDelayedPrice': 136.6, 'oddLotDelayedPriceTime': 1638123220317, 'extendedPrice': 140.01, 'extendedChange': 0.06, 'extendedChangePercent': 0.00044, 'extendedPriceTime': 1664610755498, 'previousClose': 137.45, 'previousVolume': 66163427, 'change': 0.24, 'changePercent': 0.00186, 'volume': 623

In [34]:
from datetime import datetime

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [67]:
df = pd.DataFrame(columns=list(data.keys()))
df = df.append(pd.Series(list(data.values()), index=list(data.keys())), ignore_index=True)
df

,symbol,companyName,primaryExchange,calculationPrice,open,openTime,openSource,close,closeTime,closeSource,high,highTime,highSource,low,lowTime,lowSource,latestPrice,latestSource,latestTime,latestUpdate,latestVolume,iexRealtimePrice,iexRealtimeSize,iexLastUpdated,delayedPrice,delayedPriceTime,oddLotDelayedPrice,oddLotDelayedPriceTime,extendedPrice,extendedChange,extendedChangePercent,extendedPriceTime,previousClose,previousVolume,change,changePercent,volume,iexMarketPercent,iexVolume,avgTotalVolume,iexBidPrice,iexBidSize,iexAskPrice,iexAskSize,iexOpen,iexOpenTime,iexClose,iexCloseTime,marketCap,peRatio,week52High,week52Low,ytdChange,lastTradeTime,isUSMarketOpen
0,AAPL,Apple Inc,K/L(AQGA NEAAT SE)DNCT MOLLSGEBSR,close,137.41,1651749304764,ailficof,138.06,1648882316378,iflaocfi,140.2,1638972723839,taIde EXLas Tr,135.578,1617036403679,audep5emryleed1t ci in,135.89,Close,"February 12, 2021",1615378590936,62262910,None,None,None,140.06,1678872331375,136.6,1638123220317,140.01,0.06,0.00044,1664610755498,137.45,66163427,0.24,0.00186,62394527,None,None,104178854,None,None,None,None,136.49,1662297394412,141.64,1664754176532,2281100228966,37.12,148.65,54.14,0.024637,1638092574179,False


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [82]:
stocks['Ticker'][2:10]

2     AAP
3    AAPL
4    ABBV
5     ABC
6    ABMD
7     ABT
8     ACN
9    ADBE
Name: Ticker, dtype: object

In [83]:
for stock in stocks['Ticker'][2:5]:
    api_irl =  f'https://sandbox.iexapis.com/stable/{stock}/aapl/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_irl).json()
    df = df.append(df.append(pd.Series(list(data.values()), index=list(data.keys())), ignore_index=True))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [77]:
data

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'SLR(NEA  OBGSTQEGDC)NA/KELSTLM AA', 'calculationPrice': 'close', 'open': 135.36, 'openTime': 1693011775608, 'openSource': 'lfiocaif', 'close': 140.82, 'closeTime': 1659357888944, 'closeSource': 'alociffi', 'high': 138.1, 'highTime': 1660639798908, 'highSource': 'EaIa sred TtXL', 'low': 136.589, 'lowTime': 1678074254260, 'lowSource': 'ceeiiu ade d5lmer 1ptyn', 'latestPrice': 140.38, 'latestSource': 'Close', 'latestTime': 'February 12, 2021', 'latestUpdate': 1637497455838, 'latestVolume': 62953449, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 142.12, 'delayedPriceTime': 1623092130470, 'oddLotDelayedPrice': 141.5, 'oddLotDelayedPriceTime': 1614282439658, 'extendedPrice': 136.93, 'extendedChange': 0.06, 'extendedChangePercent': 0.00045, 'extendedPriceTime': 1669462115123, 'previousClose': 135.91, 'previousVolume': 67231421, 'change': 0.25, 'changePercent': 0.00183, 'volume': 606

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

## Saving Our Excel Output

Saving our Excel file is very easy: